# TP Intelligence Artificielle - Recherche Arborescente Non Informée

# **Partie 0 : Visualisation des états**

## Voici une fonction pour visualiser les états. Nous l'utiliserons plus tard.

In [1]:
from IPython.display import display, HTML

def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile
            else:
                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

# **Partie 1 : Modélisation**

### Nous allons créer deux classes pour modeliser le taquin en espace d'états.

1. **Taquin** : Cette classe représente le problème du jeu du Taquin. Elle contient :

  * Attributs:

      * initial_state : L'état initial du Taquin, représenté par une liste de listes. Chaque sous-liste représente une ligne du Taquin, et chaque élément de la sous-liste représente une tuile. La tuile vide est représentée par le chiffre 0.
      * goal_state : L'état but du Taquin, représenté de la même manière que l'état initial.
      * size : La taille du Taquin (par exemple, 3 pour un Taquin 3x3, 4 pour un Taquin 4x4).

  * Méthodes:

      * actions(state) : Cette méthode prend un état du Taquin en entrée et retourne une liste des actions possibles à partir de cet état. Les actions possibles sont "haut", "bas", "gauche" et "droite", représentant les mouvements possibles de la tuile vide.
      * result(state, action) : Cette méthode prend un état et une action en entrée et retourne le nouvel état du Taquin après avoir appliqué l'action à l'état.
      * is_goal(state) : Cette méthode prend un état en entrée et retourne True si cet état est l'état but, False sinon.
      * cost(state, action) : Cette méthode retourne le coût de l'application d'une action à un état donné. Dans le cas du Taquin, le coût est généralement constant et égal à 1 pour chaque action.
  
2.   **Node** :  Cette classe représentera un nœud dans l'arbre de recherche. Ces attributs sont :

  * state : L'état représenté par ce nœud.
  * parent : Un pointeur vers le nœud parent (None pour le nœud racine).
  * action : L'action qui a conduit à ce nœud à partir du nœud parent (None pour le nœud racine).
  * path_cost : Le coût total du chemin depuis le nœud racine jusqu'à ce nœud (facultatif, pour les algorithmes avec des considérations de coût).
  * depth : La profondeur de ce nœud dans l'arbre (facultatif, pour la recherche en profondeur limitée).



--------------------------------------------------------------------------------
## **1.1 Classe Taquin**
--------------------------------------------------------------------------------

In [2]:
class Taquin:
    """
    A class representing the Taquin problem.
    """

    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size

    def actions(self, state):
        """Returns the possible actions (moves) from the given state."""
        # Find the position of the blank tile (0)
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Define possible moves (up, down, left, right)
        possible_actions = []
        if row > 0:
            possible_actions.append("up")
        if row < self.size-1:
            possible_actions.append("down")
        if col > 0:
            possible_actions.append("left")
        if col < self.size-1:
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """Returns the state that results from applying the given action."""
        # Create a copy of the state to avoid modifying the original
        new_state = [list(row) for row in state]

        # Find the position of the blank tile (0)
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Apply the action to move the blank tile
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):
        return state == self.goal_state  # Directly compare with goal_state

    def cost(self, state, action):
        return 1  # Default cost is 1

## **Exercise 1**
1. Créez un jeu de taquin 4x4 avec un état initial et un état objectif, puis visualisez ces deux états.
2. Identifiez les actions possibles à partir de l'état initial.
3. Appliquez une des actions possibles et visualisez le nouvel état.


In [6]:
# Define the initial and goal states
initial_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]]
print("Initial state:")
visualize_state(initial_state)

goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
print("Goal state:")
visualize_state(goal_state)

# Create a Taquin instance
problem = Taquin(initial_state, goal_state, len(initial_state[0]))

# Get possible actions from the initial state
actions = problem.actions(initial_state)
print("Possible actions:", actions)

# Apply an action to get the resulting state
new_state = problem.result(initial_state, 'up')
print("Applied action `up`. The new state is")
visualize_state(new_state)

Initial state:


1,2,3,4
5,6,7,8
9,10,11,12
13,14,,15


Goal state:


1,2,3,4
5,6,7,8
9,10,11,12
13,14,15,


Possible actions: ['up', 'left', 'right']
Applied action `up`. The new state is


1,2,3,4
5,6,7,8
9,10,,12
13,14,11,15


--------------------------------------------------------------------------------
## **1.2 Classe Node**
--------------------------------------------------------------------------------

In [3]:
class Node:
    """
    A node in a search tree.
    __init__: Initializes a node with its state, parent, action, path cost, and depth.
    __repr__: Provides a string representation of the node.
    __lt__: Defines a comparison operator for nodes based on their states.
    expand: Generates child nodes by applying all possible actions.
    child_node: Creates a single child node for a given action.
    solution: Returns the sequence of actions that led to this node.
    path: Returns the path from the root to this node as a list of nodes.
    """

    def __init__(self, state, parent=None, action=None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1

    # def __repr__(self):
    #     return "<Node {}>".format(self.state)

    # def __lt__(self, other):
    #     return self.state < other.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [
            self.child_node(problem, action)
            for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """Create a child node by applying the given action."""
        next_state = problem.result(self.state, action)
        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost = self.path_cost + problem.cost(self.state, action),
        )
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

## **Exercise 2** 
1. Créez un jeu de taquin 3x3 avec un état initial et un état objectif.
2. Créez un nœud représentant l'état initial du jeu.
3. Visualisez les enfants de l'état initial.
4. Pour chaque enfant, imprimez :
    * L'action à effectuer pour passer de l'état initial à cet enfant.
    * Le coût associé à cette action.
5. Répétez toutes les étapes pour un jeu de taquin 4x4.


In [9]:
print('------------- Taquin 3x3 -------------')
# Define the initial and goal states
initial_state = [[1, 2, 3], [4, 0, 5], [7, 8, 6]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Create a Taquin instance
problem = Taquin(initial_state, goal_state, len(initial_state[0]))

# Create the initial node
initial_node = Node(initial_state)

# Visualize the initial state
print("Initial state:")
visualize_state(initial_node.state)

# Expand the initial node to get its children
children = initial_node.expand(problem)
print("\nChildren nodes:")
for child in children:
    print(" ")
    visualize_state(child.state)
    print("Action to arrive to that node :",child.action)
    print("Path cost:", child.path_cost)


print('------------- Taquin 4x4 -------------')
# Define the initial and goal states
initial_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 11], [13, 14, 15, 12]]
goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

# Create a Taquin instance
problem = Taquin(initial_state, goal_state, len(initial_state[0]))

# Create the initial node
initial_node = Node(initial_state)

# Visualize the initial state
print("Initial state:")
visualize_state(initial_node.state)

# Expand the initial node to get its children
children = initial_node.expand(problem)
print("\nChildren nodes:")
for child in children:
    print(" ")
    visualize_state(child.state)
    print("Action to arrive to that node :",child.action)
    print("Path cost:", child.path_cost)


------------- Taquin 3x3 -------------
Initial state:


1,2,3
4,,5
7,8,6



Children nodes:
 


1,,3
4,2,5
7,8,6


Action to arrive to that node : up
Path cost: 1
 


...

Action to arrive to that node : down
Path cost: 1
 


1,2,3,4
5,6,7,8
9,,10,11
13,14,15,12


Action to arrive to that node : left
Path cost: 1
 


1,2,3,4
5,6,7,8
9,10,11,
13,14,15,12


Action to arrive to that node : right
Path cost: 1


# **Partie 2 : BFS et DFS**

Dans cette 2ème partie nous allons coder les algorithmes de recherche arborescente que nous avons étudiés en cours.

## **2.1 Breadth First Search**

### **Exercice 3** 

Créer une fonction BFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la frontière et l'ensemble des nœuds déjà explorés :
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* L'ensemble des nœuds explorés peut être une liste d'états.
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le premier élément de la liste.

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.

In [4]:
def breadth_first_search(problem, max_it = float('inf')):
    initial_node = Node(problem.initial_state)
    frontier = [initial_node]  # Use a regular list for the frontier
    explored = []  # List to keep track of explored states
    it = 0
    while frontier and it < max_it:
        it = it + 1
        node = frontier.pop(0)  # Pop the first element (index 0)
        explored.append(node.state)

        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if child_state not in explored :
                child_node = Node(child_state, parent=node, action=action)
                if problem.is_goal(child_node.state):
                      return child_node, it  # Goal found
                if child_node not in frontier:
                      frontier.append(child_node)

    return None, it  # No solution found

### **Exercice 4**
1. Appliquez votre fonction BFS pour résoudre un Taquin 3x3 avec l'état initial
   [[1, 2, 3], [4, 5, 6], [0, 7, 8]].
   Imprimez :
    * Le nombre de nœuds explorés par l'algorithme.
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * Les états successifs du chemin (utilisez la fonction visualize_state).
3. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].

In [18]:
# Define the initial and goal states
initial_state = [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Create a Taquin instance
problem = Taquin(initial_state, goal_state, len(initial_state[0]))

# Run BFS to find a solution
solution_node, it = breadth_first_search(problem)

# Print the solution (if found)
if solution_node:
    print("Solution found in",it,"iterations")
    print("Actions:", solution_node.solution())
    # Visualize the solution path (optional)
    t = 1
    for node in solution_node.path():
        print(t,'-th element of the path')
        visualize_state(node.state)
        t = t + 1
else:
    print("No solution found.")

Solution found in 1846 iterations
Actions: ['down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
1 -th element of the path


1,2,3
4,5,
6,7,8


2 -th element of the path


1,2,3
4,5,8
6,7,


3 -th element of the path


...

1,2,3
4,5,6
,7,8


13 -th element of the path


1,2,3
4,5,6
7,,8


14 -th element of the path


1,2,3
4,5,6
7,8,


## **2.2 Depth First Search**

### **Exercice 5**
Créer une fonction DFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la frontière et l'ensemble des nœuds déjà explorés :
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* L'ensemble des nœuds explorés peut être une liste d'états.
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le dernier élément de la liste.

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.



In [5]:
def depth_first_search(problem, max_it = float('inf')):
    """Performs depth-first search with a list for explored states."""
    initial_node = Node(problem.initial_state)
    frontier = [initial_node]  # Stack to represent the frontier
    explored = []  # List to keep track of explored states
    it = 0
    while frontier and it < max_it:
        it = it + 1

        node = frontier.pop(-1)
        explored.append(node.state)

        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if child_state not in explored:  # Check if child_state is in explored list
                child_node = Node(child_state, parent=node, action=action)
                if problem.is_goal(child_node.state):
                    return child_node , it  # Goal found
                if child_node not in frontier:
                    frontier.append(child_node)

    return None, it  # No solution found

### **Exercise 6**
1. Appliquez votre fonction DFS pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]]. Imprimez :
    * Le nombre de nœuds explorés par l'algorithme.
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * Les états successifs du chemin (utilisez la fonction visualize_state).
2. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].
3. Qu'observez-vous en comparant cet algorithme avec BFS ?

In [20]:
# Define the initial and goal states
initial_state = [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Create a Taquin instance
problem = Taquin(initial_state, goal_state, len(initial_state[0]))

max_it = 10000

# Run DFS to find a solution
solution_node, it = depth_first_search(problem, max_it)

# Print the solution (if found)
if solution_node:
    print("Solution found in",it,'iterations')
    print("Actions:", solution_node.solution())
    # Visualize the solution path (optional)
    t = 1
    for node in solution_node.path():
        print(t,'-th element of the path')
        visualize_state(node.state)
        t = t + 1
else:
    print("No solution found.")

No solution found.


## **2.3 Comparaison empirique de DFS et BFS**

Nous allons comparer la performance des deux algorithmes dans plusieurs jeux de Taquin 3x3. 

### **Exercice 7**
1. Écrivez une fonction compare_algorithms qui prend en entrée :
    * Une liste d’états initiaux.
    * Un état objectif.
    * Une liste d’algorithmes.
2. La fonction doit résoudre chaque problème en exécutant chaque algorithme. Assurez-vous de fixer des budgets pour éviter que les algorithmes ne prennent trop de temps.
3. Pour chaque paire (problème, algorithme), enregistrez :
    * Le nombre de nœuds explorés.
    * La longueur du chemin trouvé entre la racine et le nœud objectif.
    * Le temps d'exécution. Pour mesurer le temps d'exécution, vous pouvez importer time et utiliser :
        * *start_time = time.time()*
        * *\# résoudre le problème*
        * *end_time = time.time()*
        * *execution_time = end_time - start_time*
4. Enregistrez les résultats dans un dictionaire.



In [6]:
import time

def compare_algorithms(initial_states, goal_state, algorithms, max_it):
    results = [[] for _ in range(len(algorithms))]
    for initial_state in initial_states:
        print('Solving state')
        visualize_state(initial_state)
        problem = Taquin(initial_state, goal_state, len(initial_state[0]))
        for k in range(len(algorithms)):
            algorithm = algorithms[k]
            print('Running', algorithm.__name__)
            start_time = time.time()
            solution_node, num_explorations = algorithm(problem, max_it)
            end_time = time.time()
            execution_time = end_time - start_time

            if solution_node:
                path_length = len(solution_node.solution())
            else:
                path_length = -1  # Indicate no solution found

            results[k].append({
                "initial_state": initial_state,
                "algorithm": algorithm.__name__,  # Get the name of the function
                "num_explorations": num_explorations,
                "path_length": path_length,
                "execution_time": execution_time
            })
    return results

### **Exercice 8**
Comparez les algorithmes BFS et DFS dans les instances suivantes du jeu de Taquin 3x3.
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

In [22]:
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],  
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],  
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],  
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],  
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],  
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],  
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
algorithms = [breadth_first_search, depth_first_search]

results = compare_algorithms(initial_states, goal_state, algorithms,max_it = 20000)
results_BFS = results[0]
results_DFS = results[1]

print('Finished')

Finished


#### Utilisez le code ci-dessous pour visualiser vos résultats

In [46]:
import pandas as pd
import numpy as np
from tabulate import tabulate ## La librairie Tabulate doit être installée. Si vous ne l'avez pas, vous pouvez simplement utiliser "print(df)" pour visualiser le tableau.

Data = []
for i in range(len(initial_states)):
    Data.append([int(i+1),results_BFS[i]['num_explorations'], results_DFS[i]['num_explorations'],results_BFS[i]['path_length'],results_DFS[i]['path_length'],
                 round(results_BFS[i]['execution_time'],2),round(results_DFS[i]['execution_time'],2)])

df = pd.DataFrame(Data, columns=["State", "Nodes explored BFS", "Nodes explored DFS", "Path length BFS", "Path length DFS", "Execution Time BFS", "Execution Time DFS"])
print(tabulate(df, headers='keys', tablefmt='pretty'))

+----+-------+--------------------+--------------------+-----------------+-----------------+--------------------+--------------------+
|    | State | Nodes explored BFS | Nodes explored DFS | Path length BFS | Path length DFS | Execution Time BFS | Execution Time DFS |
+----+-------+--------------------+--------------------+-----------------+-----------------+--------------------+--------------------+
| 0  |  1.0  |       1846.0       |      20000.0       |      13.0       |      -1.0       |        0.16        |       28.78        |
| 1  |  2.0  |        6.0         |        27.0        |       3.0       |      27.0       |        0.0         |        0.0         |
| 2  |  3.0  |        7.0         |      20000.0       |       3.0       |      -1.0       |        0.0         |       36.77        |
| 3  |  4.0  |      10911.0       |      20000.0       |      17.0       |      -1.0       |        4.13        |       41.24        |
| 4  |  5.0  |       4960.0       |      20000.0       

# **Extra. Tous les jeux de taquin ont-ils une solution ?**

En cours, nous avons dit que l'espace d'état a une taille de 9!. En réalité, seulement la moitié des configurations sont résolvables. Consultez ce [**lien**](https://fr.wikipedia.org/wiki/Taquin#Configurations_solubles_et_insolubles) ainsi que la fonction suivante pour pour davantage d'*insights*.

In [18]:
import random

def is_solvable(puzzle):
    """Check if a 3x3 Taquin puzzle is solvable."""
    flattened = [tile for row in puzzle for tile in row if tile != 0]
    inversions = sum(
        1 for i in range(len(flattened)) for j in range(i + 1, len(flattened)) if flattened[i] > flattened[j]
    )
    return inversions % 2 == 0